In [16]:
import json
import random
import requests
import time
from datetime import datetime, timedelta

from src.config import ORION_URL, NGSI_LD_ENDPOINT, headers
import src.utils  as ut

In [17]:
ut.test_connection()

Status Code: 200
Orion Version: 1.15.0-next
Orion-LD Version: 1.4.0


In [23]:
def generate_single_charging_session_entity(session_number):
    """
    Generate a single charging session test entity
    """
    base_time = datetime(2025, 1, 28, 8, 0, 0)
    session_statuses = ["ended", "active", "suspended", "preparing"]
    charging_point_prefixes = [
        "fake-1-P-",
        "fake-2-P-", 
        "fake-3-P-"
    ]
    
    # Generate a unique ID with a timestamp to avoid duplication 生成包含时间戳的唯一ID，避免重复
    timestamp = int(time.time() * 1000)  # Millisecond timestamp 毫秒时间戳
    session_id = f"{session_number:03d}_{timestamp}"
    charging_point_prefix = random.choice(charging_point_prefixes)
    charging_point_id = f"{charging_point_prefix}{random.randint(1, 20):02d}"
    status = random.choice(session_statuses)
    
    # Generate random times
    start_time_offset = timedelta(
        days=random.randint(0, 30),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59)
    )
    start_time = base_time + start_time_offset
    
    session_duration = timedelta(
        hours=random.randint(0, 8),
        minutes=random.randint(30, 59)
    )
    end_time = start_time + session_duration
    
    entity = {
        "id": f"urn:ngsi-ld:ChargingSessionFake:session-{session_id}",
        "type": "ChargingSessionFake",
        
        "refChargingPoint": {
            "type": "Relationship",
            "object": f"urn:ngsi-ld:ChargingPoint:{charging_point_id}"
        },
        
        "sessionId": {
            "type": "Property",
            "value": session_id
        },
        
        "sessionStatus": {
            "type": "Property",
            "value": status
        },
        
        "sessionStartTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        },
        
        "sessionEndTime": {
            "type": "Property",
            "value": {
                "@type": "DateTime",
                "@value": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
            },
            "observedAt": end_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
        }
    }
    
    return entity

def create_single_entity_with_timing(entity, orion_url="http://localhost:1026"):
    """
    Create a single entity in Orion and measure the time
    """
    headers = {
            'Content-Type': 'application/json',
            'Accept': 'application/ld+json',
            'Link': '<https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'
    }
    
    start_time = time.time()
    
    try:
        response = requests.post(
            f"{orion_url}/ngsi-ld/v1/entities",
            headers=headers,
            data=json.dumps(entity),
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        success = response.status_code == 201
        
        # 处理409错误（实体已存在）
        if response.status_code == 409:
            return False, elapsed_ms, response.status_code, "Entity already exists"
        
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        return False, elapsed_ms, "ERROR", str(e)

def delete_single_entity_with_timing(entity_id, orion_url="http://localhost:1026"):
    """
    删除单个实体并测量时间 (类似于 create_single_entity_with_timing)
    Delete a single entity and measure timing (similar to create_single_entity_with_timing)
    """
    start_time = time.time()
    
    try:
        response = requests.delete(
            f"{orion_url}/ngsi-ld/v1/entities/{entity_id}",
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        # 删除成功的状态码是204，不是201
        # Success status code for delete is 204, not 201
        success = response.status_code == 204
        
        # 处理404错误（实体不存在）
        # Handle 404 error (entity not found)
        if response.status_code == 404:
            return False, elapsed_ms, response.status_code, "Entity not found"
        
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        return False, elapsed_ms, "ERROR", str(e)

def get_test_entities(orion_url="http://localhost:1026"):

    """
    获取所有测试实体用于删除 (修改自 clear_test_entities)
    Get all test entities for deletion (modified from clear_test_entities)
    """
    try:
        # 查询所有ChargingSessionTest类型的实体
        # Query all ChargingSessionTest type entities
        response = requests.get(
            f"{orion_url}/ngsi-ld/v1/entities",
            params={"type": "ChargingSessionFake", "limit": 1000},
            headers={"Accept": "application/json"}
        )
        
        if response.status_code == 200:
            entities = response.json()
            entity_ids = [entity["id"] for entity in entities]
            
            print(f"Found {len(entity_ids)} entities for deletion")
            
            # 显示前几个实体ID作为示例 (Show first few entity IDs as examples)
            if entity_ids:
                print("Sample entity IDs:")
                for i, entity_id in enumerate(entity_ids[:3]):
                    print(f"  {i+1}. {entity_id}")
                if len(entity_ids) > 3:
                    print(f"  ... and {len(entity_ids)-3} more")
            
            return entity_ids
        else:
            print(f"Failed to query entities: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"Error querying entities: {e}")
        return []

def run_create_performance_test(iterations=20, orion_url="http://localhost:1026"):
    """
    运行创建性能测试
    Run create performance test 
    """
    results = []
    
    print(f"Starting create performance test: {iterations} iterations")
    print("=" * 60)
    
    for i in range(1, iterations + 1):
        print(f"Create {i:2d}/{iterations}: ", end="")
        
        # 生成实体 (Generate entity)
        entity = generate_single_charging_session_entity(i)
        entity_id = entity["sessionId"]["value"]  # 统一使用 entity_id
        
        # 创建实体并计时 (Create entity with timing)
        success, elapsed_ms, response_code, error_message = create_single_entity_with_timing(entity, orion_url)
        
        # 记录结果 (Record result)
        result = {
            "iteration": i,
            "entity_id": entity_id,           # 统一命名：entity_id 而不是 session_id
            "success": success,
            "elapsed_ms": elapsed_ms,
            "response_code": response_code,
            "error_message": error_message,
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        # 打印即时结果 (Print immediate result)
        status_icon = "ok" if success else "fail"
        print(f"{status_icon} {entity_id} - {elapsed_ms:.1f}ms - {response_code}")
        
        # 请求间隔 (Request interval)
        time.sleep(0.1)
    
    return results

def run_delete_performance_test(entity_ids, orion_url="http://localhost:1026"):
    """
    运行删除性能测试 
    Run delete performance test
    """
    results = []
    iterations = len(entity_ids)  # 统一使用 iterations 表示总数
    
    print(f"Starting delete performance test: {iterations} iterations")
    print("=" * 60)
    
    for i, full_entity_id in enumerate(entity_ids, 1):
        print(f"Delete {i:2d}/{iterations}: ", end="")
        
        # 提取简短的entity_id用于显示 (Extract short entity_id for display)
        entity_id = full_entity_id.split(":")[-1]  # 统一命名格式
        
        # 删除实体并计时 (Delete entity with timing)
        success, elapsed_ms, response_code, error_message = delete_single_entity_with_timing(full_entity_id, orion_url)
        
        # 记录结果 (Record result)
        result = {
            "iteration": i,                   
            "entity_id": entity_id,           
            "success": success,               
            "elapsed_ms": elapsed_ms,         
            "response_code": response_code,   
            "error_message": error_message,   
            "timestamp": datetime.now().isoformat()  
        }
        results.append(result)
        
        # 打印即时结果 (Print immediate result)
        status_icon = "ok" if success else "fail"
        print(f"{status_icon} {entity_id} - {elapsed_ms:.1f}ms - {response_code}")
        
        # 请求间隔 (Request interval)
        time.sleep(0.1)
    
    return results

def analyze_performance_results(results, operation_type="OPERATION"):
    """
    分析性能结果 (统一分析函数，支持创建和删除)
    Analyze performance results (unified analysis function for both create and delete)
    """
    if not results:
        print("No results to analyze")
        return
    
    successful_results = [r for r in results if r["success"]]
    failed_results = [r for r in results if not r["success"]]
    
    print("\n" + "=" * 60)
    print(f"{operation_type} PERFORMANCE ANALYSIS")
    print("=" * 60)
    
    total_iterations = len(results)
    success_count = len(successful_results)
    failure_count = len(failed_results)
    success_rate = (success_count / total_iterations) * 100 if total_iterations > 0 else 0
    
    print(f"Total iterations:      {total_iterations}")
    print(f"Successful:            {success_count} ({success_rate:.1f}%)")
    print(f"Failed:                {failure_count} ({100-success_rate:.1f}%)")
    
    if successful_results:
        response_times = [r["elapsed_ms"] for r in successful_results]
        
        min_time = min(response_times)
        max_time = max(response_times)
        avg_time = sum(response_times) / len(response_times)
        
        sorted_times = sorted(response_times)
        p50 = sorted_times[len(sorted_times) // 2]
        p95_index = int(len(sorted_times) * 0.95)
        p95 = sorted_times[min(p95_index, len(sorted_times) - 1)]
        
        print(f"\nResponse Time Statistics (ms):")
        print(f"Minimum:               {min_time:.1f} ms")
        print(f"Maximum:               {max_time:.1f} ms")
        print(f"Average:               {avg_time:.1f} ms")
        print(f"Median (P50):          {p50:.1f} ms")
        print(f"95th Percentile:       {p95:.1f} ms")
        

    
    if failed_results:
        print(f"\n First 3 Failed Operations:")
        for result in failed_results[:3]:
            print(f"   {result['iteration']:2d}. {result['entity_id']} - {result['response_code']}")
            if result['error_message']:
                print(f"       Error: {result['error_message'][:100]}...")

def generate_update_data():
    """
    生成用于更新实体的随机数据
    Generate random data for updating entities
    """
    session_statuses = ["ended", "active", "suspended", "preparing"]
    
    update_data = {
        "sessionStatus": {
            "type": "Property",
            "value": random.choice(session_statuses)
        }
    }
    
    return update_data

def update_single_entity_with_timing(full_entity_id, orion_url="http://localhost:1026"):
    """
    更新单个实体并测量时间
    Update a single entity and measure timing
    """
    update_data = generate_update_data()
        
    start_time = time.time()
    
    try:
        response = requests.patch(
            f"{orion_url}/ngsi-ld/v1/entities/{full_entity_id}/attrs",
            headers=headers,
            data=json.dumps(update_data),
            timeout=30
        )
        
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        
        success = response.status_code == 204
        
        if response.status_code == 404:
            return False, elapsed_ms, response.status_code, "Entity not found"
        
        return success, elapsed_ms, response.status_code, response.text
        
    except Exception as e:
        end_time = time.time()
        elapsed_ms = (end_time - start_time) * 1000
        return False, elapsed_ms, "ERROR", str(e)

def run_update_performance_test(orion_url="http://localhost:1026"):
    """
    运行更新性能测试
    Run update performance test
    """
    # 获取现有的测试实体
    try:
        response = requests.get(
            f"{orion_url}/ngsi-ld/v1/entities",
            params={"type": "ChargingSessionTest", "limit": 1000},
            headers={"Accept": "application/json"}
        )
        
        if response.status_code == 200:
            entities = response.json()
            entity_ids = [entity["id"] for entity in entities]
            
            if not entity_ids:
                print("No test entities found to update")
                return []
                
        else:
            print(f"Failed to query entities: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"Error querying entities: {e}")
        return []
    
    # 运行更新测试
    results = []
    iterations = len(entity_ids)
    
    print(f"Starting update performance test: {iterations} iterations")
    print("=" * 60)
    
    for i, full_entity_id in enumerate(entity_ids, 1):
        print(f"Update {i:2d}/{iterations}: ", end="")
        
        entity_id = full_entity_id.split(":")[-1]
        
        success, elapsed_ms, response_code, error_message = update_single_entity_with_timing(full_entity_id, orion_url)
        
        result = {
            "iteration": i,
            "entity_id": entity_id,
            "success": success,
            "elapsed_ms": elapsed_ms,
            "response_code": response_code,
            "error_message": error_message,
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        status_icon = "ok" if success else "fail"
        print(f"{status_icon} {entity_id} - {elapsed_ms:.1f}ms - {response_code}")
        
        time.sleep(0.1)
    
    return results


In [24]:
entity_ids = get_test_entities()

Found 20 entities for deletion
Sample entity IDs:
  1. urn:ngsi-ld:ChargingSessionFake:session-001_1753714856973
  2. urn:ngsi-ld:ChargingSessionFake:session-002_1753714857116
  3. urn:ngsi-ld:ChargingSessionFake:session-003_1753714857245
  ... and 17 more


In [25]:
delete_results = run_delete_performance_test(entity_ids)

Starting delete performance test: 20 iterations
Delete  1/20: ok session-001_1753714856973 - 17.8ms - 204
Delete  2/20: ok session-002_1753714857116 - 14.6ms - 204


Delete  3/20: ok session-003_1753714857245 - 20.7ms - 204
Delete  4/20: ok session-004_1753714857379 - 21.6ms - 204
Delete  5/20: ok session-005_1753714857510 - 22.4ms - 204
Delete  6/20: ok session-006_1753714857640 - 18.6ms - 204
Delete  7/20: ok session-007_1753714857769 - 14.5ms - 204
Delete  8/20: ok session-008_1753714857883 - 14.0ms - 204
Delete  9/20: ok session-009_1753714857998 - 13.1ms - 204
Delete 10/20: ok session-010_1753714858117 - 20.0ms - 204
Delete 11/20: ok session-011_1753714858231 - 19.7ms - 204
Delete 12/20: ok session-012_1753714858476 - 12.9ms - 204
Delete 13/20: ok session-013_1753714858591 - 25.3ms - 204
Delete 14/20: ok session-014_1753714858705 - 14.0ms - 204
Delete 15/20: ok session-015_1753714858819 - 21.9ms - 204
Delete 16/20: ok session-016_1753714858934 - 26.9ms - 204
Delete 17/20: ok session-017_1753714859047 - 28.0ms - 204
Delete 18/20: ok session-018_1753714859175 - 30.7ms - 204
Delete 19/20: ok session-019_1753714859303 - 13.5ms - 204
Delete 20/20: 

In [26]:
print("delete performance:")  
analyze_performance_results(delete_results)  

delete performance:

OPERATION PERFORMANCE ANALYSIS
Total iterations:      20
Successful:            20 (100.0%)
Failed:                0 (0.0%)

Response Time Statistics (ms):
Minimum:               12.9 ms
Maximum:               30.7 ms
Average:               19.3 ms
Median (P50):          19.7 ms
95th Percentile:       30.7 ms


In [27]:
test_entity = generate_single_charging_session_entity(1)
print('Generated entity:')
print(json.dumps(test_entity, indent=2))

Generated entity:
{
  "id": "urn:ngsi-ld:ChargingSessionFake:session-001_1753715849771",
  "type": "ChargingSessionFake",
  "refChargingPoint": {
    "type": "Relationship",
    "object": "urn:ngsi-ld:ChargingPoint:fake-1-P-17"
  },
  "sessionId": {
    "type": "Property",
    "value": "001_1753715849771"
  },
  "sessionStatus": {
    "type": "Property",
    "value": "preparing"
  },
  "sessionStartTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2025-02-18T11:01:00.000Z"
    },
    "observedAt": "2025-02-18T11:01:00.000Z"
  },
  "sessionEndTime": {
    "type": "Property",
    "value": {
      "@type": "DateTime",
      "@value": "2025-02-18T19:50:00.000Z"
    },
    "observedAt": "2025-02-18T19:50:00.000Z"
  }
}


In [28]:
create_results = run_create_performance_test(20)  

Starting create performance test: 20 iterations
Create  1/20: ok 001_1753715849838 - 29.2ms - 201
Create  2/20: ok 002_1753715849968 - 19.2ms - 201
Create  3/20: ok 003_1753715850089 - 19.8ms - 201
Create  4/20: ok 004_1753715850209 - 19.5ms - 201
Create  5/20: ok 005_1753715850329 - 15.2ms - 201
Create  6/20: ok 006_1753715850445 - 20.6ms - 201
Create  7/20: ok 007_1753715850566 - 22.8ms - 201
Create  8/20: ok 008_1753715850690 - 33.7ms - 201
Create  9/20: ok 009_1753715850824 - 18.4ms - 201
Create 10/20: ok 010_1753715850945 - 20.6ms - 201
Create 11/20: ok 011_1753715851066 - 27.3ms - 201
Create 12/20: ok 012_1753715851194 - 26.4ms - 201
Create 13/20: ok 013_1753715851321 - 26.8ms - 201
Create 14/20: ok 014_1753715851449 - 24.7ms - 201
Create 15/20: ok 015_1753715851574 - 38.0ms - 201
Create 16/20: ok 016_1753715851713 - 19.8ms - 201
Create 17/20: ok 017_1753715851834 - 16.4ms - 201
Create 18/20: ok 018_1753715851951 - 21.0ms - 201
Create 19/20: ok 019_1753715852073 - 21.7ms - 201
Cr

In [29]:
print("\ncreate proformance:")
analyze_performance_results(create_results)


create proformance:

OPERATION PERFORMANCE ANALYSIS
Total iterations:      20
Successful:            20 (100.0%)
Failed:                0 (0.0%)

Response Time Statistics (ms):
Minimum:               15.2 ms
Maximum:               38.0 ms
Average:               23.0 ms
Median (P50):          21.0 ms
95th Percentile:       38.0 ms


In [30]:
entity_ids = get_test_entities()

Found 20 entities for deletion
Sample entity IDs:
  1. urn:ngsi-ld:ChargingSessionFake:session-001_1753715849838
  2. urn:ngsi-ld:ChargingSessionFake:session-002_1753715849968
  3. urn:ngsi-ld:ChargingSessionFake:session-003_1753715850089
  ... and 17 more


In [31]:
update_results = run_update_performance_test()

No test entities found to update


In [32]:
print("\nupdate proformance:")
analyze_performance_results(update_results)


update proformance:
No results to analyze
